In [1]:
## import libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from torchvision import datasets
import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
from torch.utils.data import Subset
from torch import optim

In [2]:
# check for GPU compatibility
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Load `PUBLIC` and `PRIVATE` datasets with `torchvision`


**`PRIVATE DATA`**: **train_data**; This is the dataset we want to anonymize. 10 independent teacher models will be trained on 10 different partitions of this dataset

**`PUBLIC DATA`**: **test_data**; This is the dataset upon which we will assign labels; then retrain a model on top of it and then predict on its subset.

In [3]:
# batch size
batch_size = 64

# apply data transformation strategies
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

# download and transform training and test sets
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

## PREPARE PRIVATE DATA FOR 10 DIFFERENT TEACHERS/MODELS

In [4]:
##Divide training data among teachers

# number of teachers
num_teachers = 100

teacher_loaders = []
data_size = len(train_data) // num_teachers

# iterate
for i in range(num_teachers):
    indices = list(range(i*data_size, (i+1) *data_size))
    subset_data = Subset(train_data, indices)
    loader = torch.utils.data.DataLoader(subset_data, batch_size=batch_size, num_workers=2)
    teacher_loaders.append(loader)

In [5]:
## DL model

class FCN_bn(nn.Module):
    
    def __init__(self):
        super(FCN_bn, self).__init__()
        self.model = nn.Sequential(
                  nn.Linear(784, 512),
                  nn.BatchNorm1d(512),
                  nn.ReLU(),
                  nn.Linear(512, 100),
                  nn.BatchNorm1d(100),
                  nn.ReLU(),
                  nn.Linear(100, 10))
    
    
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.model(x)
        return x

In [6]:
## TRAINING DL MODEL

def train(model, trainloader, criterion, optimizer, epochs=10):
    steps = 0
    running_loss = 0
    for e in range(epochs):
        # Model in training mode, dropout is on
        model.train()
        # iterate over batch
        for images, labels in trainloader:
            # migrate to GPU
            images, labels = images.to(device), labels.to(device)
            # zero previous gradients
            optimizer.zero_grad()
            # predict
            output = model.forward(images)
            # calculate loss
            loss = criterion(output, labels)
            # backpropagate
            loss.backward()
            # weight update
            optimizer.step()
            running_loss += loss.item()

In [7]:
## TRAINING ALL TEACHER MODELS

def train_models(num_teachers):
    models = []
    for t in range(num_teachers):
        print("Training teacher {}".format(t+1))
        model = FCN_bn()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.003)
        train(model, teacher_loaders[t], criterion, optimizer)
        models.append(model)
    return models

models = train_models(num_teachers)

Training teacher 1
Training teacher 2
Training teacher 3
Training teacher 4
Training teacher 5
Training teacher 6
Training teacher 7
Training teacher 8
Training teacher 9
Training teacher 10
Training teacher 11
Training teacher 12
Training teacher 13
Training teacher 14
Training teacher 15
Training teacher 16
Training teacher 17
Training teacher 18
Training teacher 19
Training teacher 20
Training teacher 21
Training teacher 22
Training teacher 23
Training teacher 24
Training teacher 25
Training teacher 26
Training teacher 27
Training teacher 28
Training teacher 29
Training teacher 30
Training teacher 31
Training teacher 32
Training teacher 33
Training teacher 34
Training teacher 35
Training teacher 36
Training teacher 37
Training teacher 38
Training teacher 39
Training teacher 40
Training teacher 41
Training teacher 42
Training teacher 43
Training teacher 44
Training teacher 45
Training teacher 46
Training teacher 47
Training teacher 48
Training teacher 49
Training teacher 50
Training 

In [8]:
models

[FCN_bn(
   (model): Sequential(
     (0): Linear(in_features=784, out_features=512, bias=True)
     (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): ReLU()
     (3): Linear(in_features=512, out_features=100, bias=True)
     (4): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (5): ReLU()
     (6): Linear(in_features=100, out_features=10, bias=True)
   )
 ), FCN_bn(
   (model): Sequential(
     (0): Linear(in_features=784, out_features=512, bias=True)
     (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): ReLU()
     (3): Linear(in_features=512, out_features=100, bias=True)
     (4): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (5): ReLU()
     (6): Linear(in_features=100, out_features=10, bias=True)
   )
 ), FCN_bn(
   (model): Sequential(
     (0): Linear(in_features=784, out_features=512, bias=True)
     (1): Batc

## Public Dataset

In [9]:
## Data will be labelled with laplacian noise
## New DL model will be trained on this data
student_train_data = Subset(test_data, list(range(9000)))
student_train_loader = torch.utils.data.DataLoader(student_train_data, batch_size=batch_size, 
            num_workers=4)


## TEST DATA WHERE WE MAKE FINAL PREDICTIONS
student_test_data = Subset(test_data, list(range(9000, 10000)))
student_test_loader = torch.utils.data.DataLoader(student_test_data, batch_size=batch_size, 
            num_workers=4)

In [10]:
## PREDICT WITH DL MODEL

def predict(model, dataloader):
    
    outputs = torch.zeros(0, dtype=torch.long).to(device)
    # migrate to GPU
    model.to(device)
    # set to evaluation mode
    model.eval()
    # iterate over batch
    for images, labels in dataloader:
        # migrate to GPU
        images, labels = images.to(device), labels.to(device)
        # predict
        output = model.forward(images)
        # pick up index with max. value
        ps = torch.argmax(torch.exp(output), dim=1)
        # append to "outputs" torch array
        outputs = torch.cat((outputs, ps))
    
    return outputs

## ADD LAPLACIAN NOISE AND AGGREGRATE MODELS

STEPS:
- Make predictions for each of the 10 models on the unlabelled public data i.e. `student_train_loader`. 
- Find out predicted labels and add **laplacian noise** for a certain **epsilon-delta** constraint for generating new labels.

In [11]:
# list to store predictions on unlabelled public data
preds = torch.torch.zeros((len(models), 9000), dtype=torch.long)

# iterate
for i, model in enumerate(models):
    # predict on unlabelled public data
    pred = predict(model, student_train_loader)
    preds[i] = pred

    

# list to store new labels post laplacian noise addition
labels = []


for pred in preds.transpose(1,0):
    
    # value counts for every label
    label_counts = np.bincount(pred, minlength=10)
    # calculate beta
    beta = 1 / 0.01                                    # (epsilon=0.1)
    # iterate to add laplacian noise
    for i in range(len(label_counts)):
        label_counts[i] += np.random.laplace(0, beta, 1)
    
    # catch image with max occurence
    new_label = np.argmax(label_counts)
    # add to list
    labels.append(new_label)

In [12]:
labels = np.array(labels)
preds = preds.numpy()

## Perform PATE Analysis

In [13]:
from syft.frameworks.torch.differential_privacy import pate

data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=labels, noise_eps=0.1, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 371.5129254649703
Data Dependent Epsilon: 52.36053410692698
